In [8]:
from lxml import html
import requests
from string import ascii_lowercase
import sys
import json
from xml.etree.ElementTree import tostring
from IPython.display import clear_output
from time import sleep

# base URL
page_root_url = 'http://www.jokebuddha.com/'
# file name for the topics
topics_filename = 'topicsBuddah.json'
# filename for the jokes
jokes_filename = 'jokesBuddah.json'

In [15]:
topics = {}
i = 't'
page_topic_url = page_topic_base_url + i.upper()
page_response = requests.get(page_topic_url, stream=True)
tree = html.fromstring(page_response.content)
all_topics_for_letter = tree.xpath('//div[@class="topicsList"]//text()')
for topic_for_letter in all_topics_for_letter:
    if len(topic_for_letter) > 1:
        topics[topic_for_letter] = page_topic_url
        print(page_topic_url + ': ' + topic_for_letter)
        clear_output(wait=True)

print('Gathered a total of ' + str(len(topics)) + ' topics')
print(topics)

Gathered a total of 224 topics


{'T-shirt': 'http://www.jokebuddha.com/topics/T', 'Table': 'http://www.jokebuddha.com/topics/T', 'Taco': 'http://www.jokebuddha.com/topics/T', 'Tail': 'http://www.jokebuddha.com/topics/T', 'Tailor': 'http://www.jokebuddha.com/topics/T', 'Take': 'http://www.jokebuddha.com/topics/T', 'Taliban': 'http://www.jokebuddha.com/topics/T', 'Talk': 'http://www.jokebuddha.com/topics/T', 'Tall': 'http://www.jokebuddha.com/topics/T', 'Tampons': 'http://www.jokebuddha.com/topics/T', 'Tank': 'http://www.jokebuddha.com/topics/T', 'Tape': 'http://www.jokebuddha.com/topics/T', 'Taps': 'http://www.jokebuddha.com/topics/T', 'Target': 'http://www.jokebuddha.com/topics/T', 'Tarzan': 'http://www.jokebuddha.com/topics/T', 'Task': 'http://www.jokebuddha.com/topics/T', 'Taste': 'http://www.jokebuddha.com/topics/T', 'Tattoo': 'http://www.jokebuddha.com/topics/T', 'Tax': 'http://www.jokebuddha.com/topics/T', 'Taxes': 'http://www.jokebuddha.com/topics/T', 'Taxi': 'http://www.jokebuddha.com/topics/T', 'Taylor': 'htt

In [ ]:
# gather topics
page_topic_base_url = page_root_url + 'topics/'
topics = {}
for i in ascii_lowercase:
    page_topic_url = page_topic_base_url + i.upper()
    page_response = requests.get(page_topic_url, stream=True)
    tree = html.fromstring(page_response.content)
    all_topics_for_letter = tree.xpath('//div[@class="topicsList"]//text()')
    for topic_for_letter in all_topics_for_letter:
        if len(topic_for_letter) > 1:
            topics[topic_for_letter] = page_topic_url
            print(page_topic_url + ': ' + topic_for_letter + ' ' + str(len(all_topics_for_letter)))
            clear_output(wait=True)

with open(topics_filename, 'w') as fp:
    json.dump(topics, fp)
    
print('Gathered a total of ' + str(len(topics.keys())) + ' topics')
topic_names = list(topics.keys())
print(topic_names[-1])

http://www.jokebuddha.com/topics/S: System 880


In [30]:
def get_jokes(base_page_url, iter_, topic, jokes, first_joke, tot_n_jokes = 0):  
    MAX_JOKE_LENGTH = 200
    no_more_jokes = True
    
    final_page_url = base_page_url + str(iter_)
    got_request = False
    while not got_request:
        try:
            page_response = requests.get(final_page_url, stream=True)
            got_request = True
        except:
            print('Taking a nap until the website responds zzzZZZ...')
            sleep(5)
    
    tree = html.fromstring(page_response.content)    
    joke_html_texts = tree.xpath('//p[@class="joke-inner"]')
    if len(joke_html_texts) == 0:
        return no_more_jokes, jokes, tot_n_jokes
    
    for joke_html_text in joke_html_texts:
        joke_text_list = [x for x in joke_html_text.itertext()]        
        if len(joke_text_list) == 0:
            return no_more_jokes, jokes, tot_n_jokes
        formatted_joke = ' '.join(joke_text_list)
        if formatted_joke == first_joke:
            return no_more_jokes, jokes, tot_n_jokes        
        if not formatted_joke.find('Read more') == 0 and len(formatted_joke) <= MAX_JOKE_LENGTH:
            jokes[topic].append(formatted_joke)
            print('Total number of jokes: ' + str(tot_n_jokes))
            print('Topic ' + topic + ': ' + formatted_joke)
            clear_output(wait=True)
            tot_n_jokes += 1
    
    no_more_jokes = False
    return no_more_jokes, jokes, tot_n_jokes

def get_topic_jokes(topic,page_root_url,jokes, tot_n_jokes = 0):
    final_page_url = page_root_url + topic + '/recent/'
    jokes[topic] = []
    first_joke = 'NotAJoke'
    no_more_jokes, jokes, tot_n_jokes = get_jokes(final_page_url, 1, topic, jokes, first_joke, tot_n_jokes)
    if no_more_jokes:
        return jokes, no_more_jokes, tot_n_jokes
    if len(jokes[topic]) > 0:
        first_joke = jokes[topic][0]

    for i in range(100):  
        no_more_jokes, jokes, tot_n_jokes = get_jokes(final_page_url, i+2, topic, jokes, first_joke, tot_n_jokes)   
        if no_more_jokes:            
            break    
    return jokes, no_more_jokes, tot_n_jokes       

In [36]:
with open(topics_filename, 'r') as fp:
    topics = json.load(fp)  
with open(jokes_filename, 'r') as fp:
    jokes = json.load(fp)  

topic_names = list(topics.keys())
    
tot_n_jokes = 0
topics_without_jokes = []
for topic in topics:
    if topic in jokes.keys():
        n_jokes_topic = 0
        jokes_topic = jokes[topic]
        for joke in jokes_topic:
            n_jokes_topic+=1
    else:
        topics_without_jokes.append(topic)
    tot_n_jokes += n_jokes_topic

print('Total number of jokes: ' + str(tot_n_jokes))
joke_topic_names = list(jokes.keys())
curr_topic = joke_topic_names[-1]
print('Current topic: ' + curr_topic)
print('Last topic: ' + topic_names[-1])

Total number of jokes: 42804
Current topic: toy
Last topic: Zookeeper


In [37]:
# this cell will continue form tha last processed topic
# it read the jokes file and gets the current topic as the last topic in jokes dictionary keys
jokes[curr_topic] = []
topics_to_process = topic_names[topic_names.index(curr_topic):]

for topic in topics_to_process:
    sleep(3)
    curr_topic = topic
    jokes, no_more_jokes, tot_n_jokes = get_topic_jokes(topic,page_root_url,jokes,tot_n_jokes)
    with open(jokes_filename, 'w') as fp:
        json.dump(jokes, fp)

with open(jokes_filename, 'r') as fp:
    jokes = json.load(fp)  


Total number of jokes: 46730
Topic Zookeeper: Knock Knock Who's there? Zookeeper! Zookeeper who? Zookeeper away from me!
